In [1]:
# data save by date

# COVID-19 All Related articles (and tagged relation)

In [2]:
#install dependencies 
# !pip3 install -U SPARQLWrapper
# !pip3 install -U pandas
# !pip3 install -U xlwt

In [3]:
import pandas as pd

In [4]:
# for timestamp
def tNow():
    now  = pd.Timestamp.now().strftime('%Y.%m.%d_%Hh.%Mm.%S')[0:18]
    return now

In [5]:
#getting seed based on what links to  coronavirus disease 2019 (Q84263196)  in Wikidata
import requests

whatLinks = []

# COVID -19
url = 'https://www.wikidata.org/w/api.php?action=query&format=json&list=backlinks&bltitle=Q84263196&bllimit=500&blnamespace=0'
response = requests.get(url=url).json()
whatLinks.extend(response['query']['backlinks'])

while 'continue' in response:
    url = url + '&blcontinue='+ response['continue']['blcontinue']
    response = requests.get(url=url).json()
    whatLinks.extend(response['query']['backlinks'])

QswhatLinks = [v['title'] for v in whatLinks]
QswhatLinks = set(QswhatLinks)

In [6]:
from SPARQLWrapper import SPARQLWrapper, JSON
import pandas as pd

sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
#https://w.wiki/KvX (Thanks User:Dipsacus_fullonum)
# All statements with item, property, value and rank with COVID-19 (Q84263196) as value for qualifier.

sparql.setQuery("""
SELECT ?item ?itemLabel ?property ?propertyLabel ?value ?valueLabel ?rank ?qualifier ?qualifierLabel
WHERE
{
  ?item ?claim ?statement.
  ?property wikibase:claim ?claim.
  ?property wikibase:statementProperty ?sprop.
  ?statement ?sprop ?value.
  ?statement wikibase:rank ?rank. 
  ?statement ?qprop wd:Q84263196. # COVID-19

  
  ?qualifier wikibase:qualifier ?qprop.
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}
""")
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

allStatements = pd.io.json.json_normalize(results['results']['bindings'])

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:27: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


In [7]:
# what this is for?
allStatements['valueLabel.value'].value_counts()

disease outbreak                 499
human                              7
treatment                          2
drug repositioning                 1
social stigma                      1
medical diagnosis                  1
vaccine                            1
mascot character                   1
pneumonia                          1
drug development                   1
pandemic                           1
diagnostic test                    1
hierarchy of hazard controls       1
moe anthropomorphic character      1
2020-03-05T00:00:00Z               1
epidemiological surveillance       1
epidemic                           1
Name: valueLabel.value, dtype: int64

In [8]:
# All truthy statements with COVID-19 (Q84263196) as value. --------what is truthy?
#https://w.wiki/KvZ (Thanks User:Dipsacus_fullonum)

sparql.setQuery("""
SELECT ?item ?itemLabel ?property ?propertyLabel
WHERE
{
  ?item ?claim wd:Q84263196.
  ?property wikibase:directClaim ?claim.
   SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}""")
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

truthy = pd.io.json.json_normalize(results['results']['bindings'])

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:15: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  from ipykernel import kernelapp as app


In [9]:
truthyQ = [ link.split('/')[-1] for link in truthy['item.value'].tolist()]
allStatementsQ = [ link.split('/')[-1] for link in allStatements['item.value'].tolist()]
allSPARQL = set(truthyQ).union(set(allStatementsQ))

In [10]:
allSPARQL - QswhatLinks

{'L253474-S1'}

In [11]:
QswhatLinks - allSPARQL 


{'Q10304982', 'Q66777139', 'Q84420257', 'Q85110277', 'Q88019029', 'Q88870103'}

In [12]:
Q84263196AllItems  = allSPARQL.union(QswhatLinks)

# 2019–20 COVID-19 pandemic (Q81068910)

In [13]:
# 2019–20 COVID-19 pandemic 

url = 'https://www.wikidata.org/w/api.php?action=query&format=json&list=backlinks&bltitle=Q81068910&bllimit=500&blnamespace=0'
response = requests.get(url=url).json()
whatLinks.extend(response['query']['backlinks'])

while 'continue' in response:
    url = url + '&blcontinue='+ response['continue']['blcontinue']
    response = requests.get(url=url).json()
    whatLinks.extend(response['query']['backlinks'])

QswhatLinks2 = [v['title'] for v in whatLinks]
QswhatLinks2 = set(QswhatLinks)

In [14]:
#All truthy statements with 2019–20 COVID-19 pandemic (Q81068910) as value.
#https://w.wiki/Kvd (Thanks User:Dipsacus_fullonum)

sparql.setQuery("""
# 
SELECT ?item ?itemLabel ?property ?propertyLabel WHERE {
  ?item ?claim wd:Q81068910. #2019–20 COVID-19 pandemic
  ?property wikibase:directClaim ?claim.
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}
""")
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

Q81068910 = pd.io.json.json_normalize(results['results']['bindings'])

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:15: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  from ipykernel import kernelapp as app


In [15]:
Q81068910SPARQL = [ link.split('/')[-1] for link in Q81068910['item.value'].tolist()]
Q81068910SPARQL = set(Q81068910SPARQL)

In [16]:
len(QswhatLinks2 - Q81068910SPARQL)

1284

In [17]:
len(Q81068910SPARQL - QswhatLinks2)

541

In [18]:
Q81068910All = Q81068910SPARQL.union(QswhatLinks2)

In [19]:
# Joining the full set from both seeds
Qs = Q81068910All.union(Q84263196AllItems)

In [20]:
## add seeds (by definition they were out)
Qs = Q81068910All.union({'Q81068910','Q84263196'})

In [21]:
len(Qs)

3000

In [22]:
# Adding a third seed  SARS-CoV-2 (Q82069695)
url = 'https://www.wikidata.org/w/api.php?action=query&format=json&list=backlinks&bltitle=Q82069695&bllimit=500&blnamespace=0'
response = requests.get(url=url).json()
whatLinks.extend(response['query']['backlinks'])

while 'continue' in response:
    url = url + '&blcontinue='+ response['continue']['blcontinue']
    response = requests.get(url=url).json()
    whatLinks.extend(response['query']['backlinks'])

QswhatLinks3 = [v['title'] for v in whatLinks]
QswhatLinks3 = set(QswhatLinks)

In [23]:
Qs = Qs.union(QswhatLinks3) 

In [24]:
len(Qs)

3000

# Getting articles and relationship

In [25]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]
        
        
wikidata_query_base = 'https://www.wikidata.org/w/api.php?action=wbgetentities&format=json&ids=' 
itemsInfo = {}
c = 0
for items in chunks(list(Qs),50):
    c +=50
    if c%200 ==0: print(c,'items reviewed')
    url = wikidata_query_base  + '|'.join(items)
    itemsInfo.update(requests.get(url=url).json()['entities'])

200 items reviewed
400 items reviewed
600 items reviewed
800 items reviewed
1000 items reviewed
1200 items reviewed
1400 items reviewed
1600 items reviewed
1800 items reviewed
2000 items reviewed
2200 items reviewed
2400 items reviewed
2600 items reviewed
2800 items reviewed
3000 items reviewed


In [26]:

def getRelationships(claims,targetQs):
    '''
    This function receives a list of claims from a Wikidata Item, and a list of target Qs
    Iterating over the claims, looking for the target Qs and returning the pair Property and target Q
    For example, if it find relationship Part of (P31) of Q12323 (that is the target list)
    will return [(P31,Q3)]
    inputs:
    claims: object, result from wikidata queries like 
            'https://www.wikidata.org/w/api.php?action=wbgetentities&format=json&ids=Q5' 
    targetQs: list of str, where str are Q values 
    output:
        return a list of pairs (prop,target)
    '''
    pairs = []
    for prop, relationships in claims.items():
        for relationship in relationships:
            if 'mainsnak' in relationship:
                datatype = relationship['mainsnak'].get('datatype','')
                if datatype=='wikibase-item':
                    try: #found some cases without  id even for a wikibase-item datatype
                        Qfound = relationship['mainsnak']['datavalue']['value'].get('id','')
                        if Qfound in targetQs:
                            pairs.append([prop,targetQs[targetQs.index(Qfound)]])
                    except:
                        pass
    if not pairs:
        pairs.append(['unknown','unknown'])
    return pairs

def getValueIfWikidataItem(claim):
    '''
    this function return a list of values for a given claim, if those values point to a wikidata item
    datatype=='wikibase-item'
    input:
    claim: object
    output:
    wikidataItems: list of str
      '''
    output = []
    for relationship in claim:
        if 'mainsnak' in relationship:
            datatype = relationship['mainsnak'].get('datatype','')
            if datatype=='wikibase-item':
                Qfound = relationship['mainsnak']['datavalue']['value'].get('id','')
                output.append(Qfound)
    if not output:
        output.append('unknown')
    return output

In [27]:
pagesPerProject = {}
pagesPerProjectTable = {}
itemsInfoTable = {}
labelsEn = {}
for item,v in itemsInfo.items():
    itemsInfoTable[item] = {}
    try:
        itemsInfoTable[item]['item Label'] = v['labels']['en']['value']
    except:
        itemsInfoTable[item]['item Label'] = 'unknown '
    #checking if there are claims for that Q, if not claims we return an empty dict, to avoid errors
    claims = v.get('claims',{})
    if 'P31' in  claims: #getting part of to classify the item        
        itemsInfoTable[item]['Instace Of'] = getValueIfWikidataItem(claims.get('P31'))
    else:
        itemsInfoTable[item]['Instace Of'] = ['unknown']
    #find COVID-19 / COVID-19 pandemics relationships
    itemsInfoTable[item]['RelationTuple'] = getRelationships(claims,['Q81068910','Q84263196'])

    if 'sitelinks' in v:
        for wiki,data in v['sitelinks'].items():
            page = data['title']
            project ='%s.wikipedia' % wiki.replace('wiki','')
            pagesPerProject[project] = pagesPerProject.get(project,[])
            pagesPerProject[project].append(page)
            article_link = 'https://%s.org/wiki/%s' % (project,page)
            projectcode = project.split('.')[0]
            wikilink = '[[%s:%s|%s]]' % (projectcode,page,page)
            pagesPerProjectTable[article_link] = {'project':project,'page':page,'wikidataItem':item,'wikilink':wikilink}
            
            
itemsInfoTable = pd.DataFrame.from_dict(itemsInfoTable,orient='index')

In [28]:
pagesPerProjectTable = pd.DataFrame.from_dict(pagesPerProjectTable,orient='index')

In [29]:
#FINALVERSION Of pagesPerProjectTable

pagesPerProjectTable['url'] = pagesPerProjectTable.index
pagesPerProjectTable

,project,page,wikidataItem,wikilink,url
https://arz.wikipedia.org/wiki/دال فولويل,arz.wikipedia,دال فولويل,Q5210439,[[arz:دال فولويل|دال فولويل]],https://arz.wikipedia.org/wiki/دال فولويل
https://azb.wikipedia.org/wiki/دال فولول,azb.wikipedia,دال فولول,Q5210439,[[azb:دال فولول|دال فولول]],https://azb.wikipedia.org/wiki/دال فولول
https://en.wikipedia.org/wiki/Dale Folwell,en.wikipedia,Dale Folwell,Q5210439,[[en:Dale Folwell|Dale Folwell]],https://en.wikipedia.org/wiki/Dale Folwell
https://arz.wikipedia.org/wiki/ايسابيل دياز ايوسو,arz.wikipedia,ايسابيل دياز ايوسو,Q50634772,[[arz:ايسابيل دياز ايوسو|ايسابيل دياز ايوسو]],https://arz.wikipedia.org/wiki/ايسابيل دياز ايوسو
https://ast.wikipedia.org/wiki/Isabel Díaz Ayuso,ast.wikipedia,Isabel Díaz Ayuso,Q50634772,[[ast:Isabel Díaz Ayuso|Isabel Díaz Ayuso]],https://ast.wikipedia.org/wiki/Isabel Díaz Ayuso
...,...,...,...,...,...
https://en.wikipedia.org/wiki/Ted O'Brien,en.wikipedia,Ted O'Brien,Q7693568,[[en:Ted O'Brien|Ted O'Brien]],https://en.wikipedia.org/wiki/Ted O'Brien
https://ar.wikipedia.org/wiki/جائحة فيروس كورونا في هاواي 2020,ar.wikipedia,جائحة فيروس كورونا في هاواي 2020,Q87746177,[[ar:جائحة فيروس كورونا في هاواي 2020|جائحة في...,https://ar.wikipedia.org/wiki/جائحة فيروس كورو...
https://commons.wikipedia.org/wiki/Category:COVID-19 pandemic in Hawaii,commons.wikipedia,Category:COVID-19 pandemic in Hawaii,Q87746177,[[commons:Category:COVID-19 pandemic in Hawaii...,https://commons.wikipedia.org/wiki/Category:CO...
https://en.wikipedia.org/wiki/2020 coronavirus pandemic in Hawaii,en.wikipedia,2020 coronavirus pandemic in Hawaii,Q87746177,[[en:2020 coronavirus pandemic in Hawaii|2020 ...,https://en.wikipedia.org/wiki/2020 coronavirus...


In [30]:
#Save the information
import pickle
with open(f'pagesPerProject-{tNow()}.pickle','wb') as f:
    pickle.dump(pagesPerProject,f)

pagesPerProjectTable.to_csv(f'pagesPerProjectTable-{tNow()}.csv')

# .
# Getting Pageviews 
# from (Diego's coronavirus nb & Geng's main nb) 
# added by Wook

In [31]:
### Functions to get pageviews per page
### I'm not using mwviews package for this, because that have problems with non-latin characters


def getRedirects(page,project):
    '''
    get all redirects (upto 500)
    for a given page 
    '''
    base_url = 'https://%s.org/w/api.php?action=query&titles=%s&prop=redirects&rdlimit=500&format=json' % (project,page)
    data = [p['title'] for p in list(requests.get(base_url).json()['query']['pages'].values())[0]['redirects']]
    return data
    

def getViews(page,start,end,project):
    """
    get pageviews using this API https://wikitech.wikimedia.org/wiki/Analytics/AQS/Pageviews 
    page: str (article name)
    start: str start date YYYYMMDD (20200101)
    end: str end date YYYYMMDD (20200103)
    project: str, ex: en.wikipedia (project does not include .org)

    """
    base_url = "https://wikimedia.org/api/rest_v1/metrics/pageviews/per-article/%s/all-access/all-agents/%s/daily/%s/%s" % (project,page,start,end)
    data = requests.get(base_url).json()['items']
    df = pd.DataFrame(data) [['views','timestamp']]
    df.rename(columns={'views':page},inplace=True)
    df['timestamp'] = pd.to_datetime(df['timestamp'], format='%Y%m%d%H')
    df.set_index('timestamp',inplace=True)
    return df


def getViewsWithRedirects(page,start,end,project):
    """
    Get all redirects going to 'page' and get pageviews for that article
    page: str (article's title)
    start: str start date YYYYMMDD (20200101)
    end: str end date YYYYMMDD (20200103)
    project: str, ex: en.wikipedia (project does not include .org)
    """
    #start list of page
    pages = [page]
    try:
        #get all redirects to page 
        redirects = getRedirects(page=page,project=project)     
    except:
        #if getRedirects gives an error, we assume that there no pages redirecting to page
        redirects = []
    pages.extend(redirects)
    ## get pages views for all articles
    results = getViewsMultiples(pages=pages,start=start,end=end,project=project)
    #sum all pages views
    results = pd.DataFrame(results.sum(axis=1))
    results.rename(columns={0:page},inplace=True)
    return results      

    
def getViewsMultiples(pages,start,end,project):
    """
    Get page views for a list of pages 
    pages: list (list of article's titles) ex: ['Chile','Brasil','Argentina']
    start: str start date YYYYMMDD (20200101)
    end: str end date YYYYMMDD (20200103)
    project: str, ex: en.wikipedia (project does not include .org)
    """
    results = []
    for page in pages:
        try:
            results.append(getViews(page,start,end,project))
        except:
            pass
    return pd.concat(results,axis=1)

def getViewsMultiplesWithRedirects(pages,start,end,project):
    """
    Get page views for a list of pages considering the redirects for each page
    pages: list (list of article's titles) ex: ['Chile','Brasil','Argentina']
    start: str start date YYYYMMDD (20200101)
    end: str end date YYYYMMDD (20200103)
    project: str, ex: en.wikipedia (project does not include .org)
    """
    
    results = []
    for page in pages:
        try:
            results.append(getViewsWithRedirects(page,start,end,project))
            print(page)
        except:
            pass
    return pd.concat(results,axis=1)
    



In [32]:
## Start the mwviews package to check pageviews per project
from mwviews.api import PageviewsClient
import requests
p = PageviewsClient(user_agent="research",parallelism=200)

In [33]:
#define dates for pageviews in format YYYYMMDD
dates = {'start':'20200110','end':'20200408'}

In [34]:
#Get pageviews per page
#Don't worry about warnings. They come for articles without Pageviews during the select period (there many reasons for that)
viewsCorona = {}
for project, pages in pagesPerProject.items():
    if project in ['ko.wikipedia','en.wikipedia','es.wikipedia', 'it.wikipedia', 'zh.wikipedia']:
        print(project)
        viewsCorona[project] = getViewsMultiplesWithRedirects(pages=pages,start=dates['start'],end=dates['end'],project=project)

en.wikipedia
Dale Folwell
Isabel Díaz Ayuso
Budi Karya Sumadi
Aaron Rubashkin
Timeline of the 2019–20 coronavirus pandemic in April 2020
2020 coronavirus pandemic in Delaware
Preta Gil
Joe Runions
2020 coronavirus pandemic in Myanmar
2020 coronavirus pandemic in Nigeria
Robert H. Garff
Juan Giménez
2020 coronavirus pandemic in Curaçao
Pere Aragonès
Daniele Rugani
Masoumeh Ebtekar
Igor Boev
2020 coronavirus pandemic in California
NHS Nightingale Hospitals
2020 coronavirus pandemic in Japan
Tom Dempsey
Alexander Graf Lambsdorff
Luigi Roni
2020 coronavirus pandemic in the Canary Islands
Health Protection (Coronavirus, Restrictions) (England) Regulations 2020
Rodolfo González Rissotto
José María Loizaga Viguri
2020 coronavirus pandemic in Kenya
Clemens Pickel
2020 coronavirus pandemic in Finland
2020 coronavirus pandemic in Andorra
Dominique de Legge
2020 coronavirus pandemic in Nauru
Mirga Gražinytė-Tyla
Luis Fernández
2020 coronavirus pandemic in Cameroon
Josep Maria Benet i Jornet
Camer

Peter Dutton
2020 coronavirus pandemic in Guinea
Davi Alcolumbre
Macarena Olona Choclán
Kate Osborne
Harouna Kaboré
2019–20 coronavirus pandemic on social media
Brooke Baldwin
Families First Coronavirus Response Act
Category:2019–20 coronavirus pandemic in China
Valérie Boyer
Tocilizumab
Colton Underwood
Eliaquim Mangala
2020 coronavirus pandemic in North Korea
2020 coronavirus pandemic in the Community of Madrid
2020 coronavirus pandemic in Kyrgyzstan
Hal Willner
Sophie Grégoire Trudeau
2020 coronavirus pandemic in Bahrain
René Trabelsi
2020 coronavirus pandemic in Idaho
Helene Weinstein
Duan Zhengcheng
2020 coronavirus pandemic in Maharashtra
Rodrigo Pesántez Rodas
Hamid Kahram
Hisham Hamdan
Carmen Leyte
Jason Collins
Hagen Reinhold
Wong Ling Biu
2020 coronavirus pandemic in Argentina
Vittorio Gregotti
2020 coronavirus pandemic in the Bahamas
José Folgado
Mohamed Farah (footballer)
List of deaths from the 2019–20 coronavirus pandemic
2020 coronavirus pandemic in Lebanon
Yang Xiaobo (

2020 coronavirus pandemic in Sudan
Human rights issues during the 2019–20 coronavirus pandemic
2020 coronavirus pandemic in Saint Barthélemy
Marc Mézard
Category:2019–20 coronavirus pandemic templates
Joe Exotic
2020 coronavirus pandemic in Pakistan
2020 coronavirus pandemic in Mozambique
Rafael Gómez Nieto
Ana Pastor (politician)
Idris Elba
2020 coronavirus pandemic in the Faroe Islands
Romain Lagarde
2020 coronavirus pandemic in the Northern Mariana Islands
Template:2019–20 coronavirus pandemic
Rita Wilson
2020 coronavirus pandemic in Iraq
2020 coronavirus pandemic in Panama
Nicola Zingaretti
Eshaq Jahangiri
2020 coronavirus pandemic in Grenada
2020 coronavirus pandemic in Somalia
2020 coronavirus pandemic in the Republic of Ireland
Omar Fayad
Mohammad Ali Ramazani Dastak
Alberto Cirio
Rafael Ribó
Michael Saag
Lenni-Kim
Élisabeth Toutut-Picard
Zororo Makamba
2020 coronavirus pandemic in Africa
Media coverage of the 2019–20 coronavirus pandemic
Julie Bennett
Zaccaria Cometti
Jean-Jacq

Brett Crozier
2020 coronavirus pandemic in Papua New Guinea
2020 coronavirus pandemic in Cape Verde
2020 coronavirus pandemic in Cuba
2020 coronavirus pandemic in Montana
Pat Foye
2020 coronavirus pandemic in North Rhine-Westphalia
George Alagiah
Goyo Benito
2020 coronavirus pandemic in Vietnam
2020 coronavirus pandemic in Malaysia
Sholom Lipskar
2019–20 coronavirus pandemic in Asia
Jean-Laurent Cochet
Christian Estrosi
2020 coronavirus pandemic in Lithuania
Jean-Luc Reitzer
2020 coronavirus pandemic in Montenegro
2020 coronavirus pandemic in Egypt
2020 coronavirus pandemic in East Timor
Mostafa Pourmohammadi
COVID-19 testing
William Wolf (critic)
2019–20 coronavirus pandemic in Hubei
Fabian Nürnberger
2020 coronavirus pandemic in Madagascar
Damantang Camara
Thiago Seyboth Wild
Mike Kelly (Pennsylvania politician)
Coronavirus disease
Masataka Nashida
David Lat
2020 coronavirus pandemic in Maryland
Zhong Nanshan
Guillaume Vuilletet
2020 coronavirus pandemic in Réunion
2020 coronavirus p

Branislav Blažić
Walter Robb
Pandemia de enfermedad por coronavirus de 2020 en Australia
Martín Ortega
Giuliana De Sio
Detto Mariano
Olle Holmquist
Síndrome respiratorio agudo grave
Juan José Yúnez
Martti Ahtisaari
Muhammad Sirajul Islam
Cuarentena por la pandemia de enfermedad por coronavirus de 2019-2020
Santiago Abascal Conde
Mark Blum
Pandemia de enfermedad por coronavirus de 2020 en Noruega
Greg Rikaart
Adam Schlesinger
Li Wenliang
Fabio Depaoli
Lucien Sève
Pandemia de enfermedad por coronavirus de 2020 en América
Pandemia de enfermedad por coronavirus de 2020 en Paraguay
Pandemia de enfermedad por coronavirus de 2020 en Arabia Saudita
Sergio Bassi
Vicente Capdevila
Pandemia de enfermedad por coronavirus de 2020 en Honduras
Pandemia de enfermedad por coronavirus de 2020 en Suecia
Mads Mensah Larsen
Patrick Bruel
Sérgio Trindade
María Teresa de Borbón-Parma
Jean Leber
Emilia Currás
Sergio Rossi
Patrick Groetzki
Desinformación sobre la pandemia de coronavirus de 2019-2020
Jorge Alci

2019冠狀病毒病泰國疫情
Template:COVID-19-stub
布雷特·迪恩
凯文·杜兰特
柳帆
威爾斯親王查爾斯
娜汀·多里斯
埃爾文·恩加佩斯
安傑洛·德多納蒂斯
伊斯梅尔·纳杰尔
2019冠状病毒病开曼群岛疫情
2019冠状病毒病刚果民主共和国疫情
乔凡尼·巴蒂斯塔·拉比诺
布莱瑟·马图伊迪
2019冠状病毒病法属圭亚那疫情
2019冠状病毒病中非共和国疫情
侯赛因·谢赫伊斯兰
2019冠狀病毒病南非疫情
2019冠狀病毒病疫情全球撤僑行動
肖恩·佩顿
2019冠状病毒病澳大利亚疫情
詹姆斯·劳伦斯·多兰
嚴重急性呼吸系統綜合症
马哈茂德·萨德吉
2019冠狀病毒病捷克疫情
马尔蒂·阿赫蒂萨里
圣地亚哥·阿瓦斯卡尔
2019冠状病毒病乌干达疫情
2019冠状病毒病布基纳法索疫情
马克·布鲁姆
2019冠状病毒病挪威疫情
格雷格·里卡特
朱和平 (1953年)
亚当·施莱辛格
李文亮
吕西安·塞夫
2019冠状病毒病斯威士兰疫情
2019冠狀病毒病澳門疫情
Category:2019冠狀病毒病南美洲疫情
Category:2019冠狀病毒病相關人物
長坂拳彌
2019冠狀病毒病沙特阿拉伯疫情
2019冠状病毒病阿鲁巴疫情
Category:2019冠狀病毒病中國大陸疫情
2019冠状病毒病马约特疫情
2019冠状病毒病洪都拉斯疫情
2019冠状病毒病瑞典疫情
2019冠状病毒病文莱疫情
Wikipedia:2019冠状病毒病专题
2019冠状病毒病疫情对电视业的影响
凱莉·西蒙茲
玛丽亚·特蕾莎·德·波旁-帕尔马
塞爾焦·羅西
2019冠狀病毒病疫情相關錯誤資訊
2019冠狀病毒病全球各地疫情
2019冠状病毒病圭亚那疫情
2019冠状病毒病圣巴泰勒米疫情
Category:2019冠狀病毒病病例數模板
2019冠狀病毒病巴基斯坦疫情
拉斐尔·戈麦斯·涅托
安娜·帕斯托尔·胡利安
伊德瑞斯·艾尔巴
2019冠状病毒病法罗群岛疫情
Template:COVID-19
麗塔·威爾遜
尼古拉·津加雷蒂
埃沙格·贾汉吉里
2019冠状病毒病索马里疫情
2019冠状病毒病爱尔兰疫情
穆罕默德·阿里·拉马扎尼·达斯塔克
阿尔贝托·奇里奥
江学庆
2019冠狀病毒病武漢市疫情
2019冠状病毒病非洲疫情
Category:2019冠状病毒病意大利疫情
黄文军
塚原直

Mattia Zaccagni
Thomas Kahlenberg
Angelo Moreschi
Irene Montero
Nydia Velázquez
Patrick McEnroe
Naomi Munakata
Olga Kurylenko
YNW Melly
Thierry Dusautoir
Mahmud Gibril
Ospedale Leishenshan
Brian Stokes Mitchell
Pino Grimaldi
Marouane Fellaini
John H. Cox
Esperanza Aguirre
Jacques Oudin (politico)
Morten Thorsby
Jonathas
Javier Ortega Smith
Christopher Cross
Christian Wood
Pandemia di COVID-19 del 2020 in Europa
Matt Hancock
Dmitrij Strachov
Patrizia Barbieri
Daniel Křetínský
John Taylor (bassista)
Danilo Barozzi (ciclista)
Categoria:COVID-19
Andy Cohen
Bucky Pizzarelli
Stefan Thesker
Brune Poirson
Giovanni Blandino
Lorenzo Sanz
Nur Hassan Hussein
Raffaele Masto
Serge Telle
Pandemia di COVID-19 del 2020 in Corea del Sud
Pandemia di COVID-19 del 2020 negli Stati Uniti d'America
Maximiliano Richeze
Quim Torra
Ferdinand Marcos Jr.
Clive Allen
Mikel Arteta
Wu Lei
Charlotte Lawrence
Daniel Dae Kim
COVID-19
Ellis Marsalis
Angelo Rottoli
Martinho Lutero Galati
Sara Bareilles
Tom Hanks
José Lui

In [35]:
with open(f'ViewsEachRelatedPagePerLanguage-{tNow()}.pickle','wb') as f:
    pickle.dump(viewsCorona,f)

In [36]:
viewsCorona

{'en.wikipedia':             Dale Folwell  Isabel Díaz Ayuso  Budi Karya Sumadi  \
 timestamp                                                        
 2020-01-10          29.0               72.0                 57   
 2020-01-11          20.0               62.0                 16   
 2020-01-12         110.0              162.0                 28   
 2020-01-13          23.0              290.0                 34   
 2020-01-14          44.0              138.0                 26   
 ...                  ...                ...                ...   
 2020-04-03          89.0              213.0                102   
 2020-04-04          47.0              104.0                 64   
 2020-04-05          35.0              189.0                 74   
 2020-04-06          29.0              193.0                 77   
 2020-04-07          36.0               94.0                 64   
 
             Aaron Rubashkin  \
 timestamp                     
 2020-01-10             18.0   
 2020-01-11    

# itemsInfoTable for labels

In [37]:
itemsInfoTable = itemsInfoTable.explode('Instace Of').explode('RelationTuple')

In [38]:
itemsInfoTable['connector'] = itemsInfoTable['RelationTuple'].apply(lambda x:x[0])
itemsInfoTable['connected To'] = itemsInfoTable['RelationTuple'].apply(lambda x:x[1])
itemsInfoTable.drop('RelationTuple',inplace=True,axis=1)

In [39]:
connectedToLabel = {'Q84263196':'COVID-19', 'Q81068910':'2019–20 COVID-19 pandemic'} 
itemsInfoTable['connected To Label'] = itemsInfoTable['connected To'].apply(lambda x:connectedToLabel.get(x))

In [40]:
## Getting labels for connector (properties)
Ps = list(itemsInfoTable['connector'].unique())
props = []
for P in Ps:
    props.append(requests.get('https://www.wikidata.org/w/api.php?action=wbgetentities&ids=%s&format=json' % P).json())

In [41]:
propLabels ={}
for P in props:
    if 'entities' in P:
        for Pid,data in P['entities'].items():
            tmplabel = data.get('labels').get('en',{})
            propLabels[Pid]= tmplabel.get('value','unknown')
propLabels = pd.DataFrame.from_dict(propLabels,orient='index',columns=['connector Label'])
propLabels['connector'] = propLabels.index

In [42]:
itemsInfoTable = itemsInfoTable.join(propLabels, on='connector',rsuffix='_tmp').drop('connector_tmp',axis=1)

In [43]:
itemsInfoTable['item_id'] = itemsInfoTable.index

In [44]:
itemsInfoTable

,item Label,Instace Of,connector,connected To,connected To Label,connector Label,item_id
Q5210439,Dale Folwell,Q5,P1050,Q84263196,COVID-19,medical condition,Q5210439
Q88938180,2020 coronavirus pandemic in Swiss canton BE,Q3241045,unknown,unknown,None,NaN,Q88938180
Q50634772,Isabel Díaz Ayuso,Q5,P1050,Q84263196,COVID-19,medical condition,Q50634772
Q88976039,Role of temperature and humidity in the modula...,Q13442814,P921,Q84263196,COVID-19,main subject,Q88976039
Q88976039,Role of temperature and humidity in the modula...,Q13442814,P921,Q81068910,2019–20 COVID-19 pandemic,main subject,Q88976039
...,...,...,...,...,...,...,...
Q87746177,2020 coronavirus pandemic in Hawaii,Q3241045,P361,Q81068910,2019–20 COVID-19 pandemic,part of,Q87746177
Q88979317,SARS-CoV-2 and ORF3a: Non-Synonymous Mutations...,Q13442814,P921,Q84263196,COVID-19,main subject,Q88979317
Q88979317,SARS-CoV-2 and ORF3a: Non-Synonymous Mutations...,Q13442814,P921,Q81068910,2019–20 COVID-19 pandemic,main subject,Q88979317
Q88661507,Impact of the 2019–20 coronavirus pandemic on ...,unknown,P1269,Q81068910,2019–20 COVID-19 pandemic,facet of,Q88661507


In [45]:
## Getting Instance of labels
instaceOfQs = list(itemsInfoTable['Instace Of'].unique())
print(len(instaceOfQs))
QiOf = [] # Q instace
for Q in instaceOfQs:
    QiOf.append(requests.get('https://www.wikidata.org/w/api.php?action=wbgetentities&ids=%s&format=json' % Q).json())

98


In [46]:

QiOfLabels ={}
for P in QiOf:
    if 'entities' in P:
        for Pid,data in P['entities'].items():
            tmplabel = data.get('labels').get('en',{})
            QiOfLabels[Pid]= tmplabel.get('value','unknown')
QiOfLabels = pd.DataFrame.from_dict(QiOfLabels,orient='index',columns=['Instace Of Label'])
QiOfLabels['Instace Of'] = QiOfLabels.index

In [47]:
#FINALVERSION Of Info Table
itemsInfoTable = itemsInfoTable.join(QiOfLabels, on='Instace Of',rsuffix='_tmp').drop('Instace Of_tmp',axis=1)

In [48]:
itemsInfoTable.to_csv(f'itemsInfoTable-{tNow()}.csv')
with open(f'itemsInfoTable-{tNow()}.pickle','wb') as f:
    pickle.dump(itemsInfoTable,f)

In [49]:
itemsInfoTable

,item Label,Instace Of,connector,connected To,connected To Label,connector Label,item_id,Instace Of Label
Q5210439,Dale Folwell,Q5,P1050,Q84263196,COVID-19,medical condition,Q5210439,human
Q88938180,2020 coronavirus pandemic in Swiss canton BE,Q3241045,unknown,unknown,None,NaN,Q88938180,disease outbreak
Q50634772,Isabel Díaz Ayuso,Q5,P1050,Q84263196,COVID-19,medical condition,Q50634772,human
Q88976039,Role of temperature and humidity in the modula...,Q13442814,P921,Q84263196,COVID-19,main subject,Q88976039,scholarly article
Q88976039,Role of temperature and humidity in the modula...,Q13442814,P921,Q81068910,2019–20 COVID-19 pandemic,main subject,Q88976039,scholarly article
...,...,...,...,...,...,...,...,...
Q87746177,2020 coronavirus pandemic in Hawaii,Q3241045,P361,Q81068910,2019–20 COVID-19 pandemic,part of,Q87746177,disease outbreak
Q88979317,SARS-CoV-2 and ORF3a: Non-Synonymous Mutations...,Q13442814,P921,Q84263196,COVID-19,main subject,Q88979317,scholarly article
Q88979317,SARS-CoV-2 and ORF3a: Non-Synonymous Mutations...,Q13442814,P921,Q81068910,2019–20 COVID-19 pandemic,main subject,Q88979317,scholarly article
Q88661507,Impact of the 2019–20 coronavirus pandemic on ...,unknown,P1269,Q81068910,2019–20 COVID-19 pandemic,facet of,Q88661507,NaN


# pagesPerProject join itemsInfoTable

# Nonhumans 
# human이랑 human 아닌 거랑 구분해서 쓸 때 이용하자.

In [50]:
nonHumans  = itemsInfoTable[itemsInfoTable['Instace Of Label'] != 'human']

In [51]:
nonHumans

,item Label,Instace Of,connector,connected To,connected To Label,connector Label,item_id,Instace Of Label
Q88938180,2020 coronavirus pandemic in Swiss canton BE,Q3241045,unknown,unknown,None,NaN,Q88938180,disease outbreak
Q88976039,Role of temperature and humidity in the modula...,Q13442814,P921,Q84263196,COVID-19,main subject,Q88976039,scholarly article
Q88976039,Role of temperature and humidity in the modula...,Q13442814,P921,Q81068910,2019–20 COVID-19 pandemic,main subject,Q88976039,scholarly article
Q88978435,Covid-19 health care demand and mortality in S...,Q13442814,P921,Q84263196,COVID-19,main subject,Q88978435,scholarly article
Q88978435,Covid-19 health care demand and mortality in S...,Q13442814,P921,Q81068910,2019–20 COVID-19 pandemic,main subject,Q88978435,scholarly article
...,...,...,...,...,...,...,...,...
Q88974578,Estimation of the final size of the coronaviru...,Q13442814,P921,Q81068910,2019–20 COVID-19 pandemic,main subject,Q88974578,scholarly article
Q87746177,2020 coronavirus pandemic in Hawaii,Q3241045,P361,Q81068910,2019–20 COVID-19 pandemic,part of,Q87746177,disease outbreak
Q88979317,SARS-CoV-2 and ORF3a: Non-Synonymous Mutations...,Q13442814,P921,Q84263196,COVID-19,main subject,Q88979317,scholarly article
Q88979317,SARS-CoV-2 and ORF3a: Non-Synonymous Mutations...,Q13442814,P921,Q81068910,2019–20 COVID-19 pandemic,main subject,Q88979317,scholarly article


In [52]:
nonHumansPages = nonHumans.join(pagesPerProjectTable.set_index('wikidataItem'))

In [53]:
nonHumansPages

,item Label,Instace Of,connector,connected To,connected To Label,connector Label,item_id,Instace Of Label,project,page,wikilink,url
Q103177,severe acute respiratory syndrome,Q18123741,P1542,Q81068910,2019–20 COVID-19 pandemic,has effect,Q103177,infectious disease,af.wikipedia,Ernstige akute respiratoriese sindroom,[[af:Ernstige akute respiratoriese sindroom|Er...,https://af.wikipedia.org/wiki/Ernstige akute r...
Q103177,severe acute respiratory syndrome,Q18123741,P1542,Q81068910,2019–20 COVID-19 pandemic,has effect,Q103177,infectious disease,ar.wikipedia,متلازمة تنفسية حادة وخيمة,[[ar:متلازمة تنفسية حادة وخيمة|متلازمة تنفسية ...,https://ar.wikipedia.org/wiki/متلازمة تنفسية ح...
Q103177,severe acute respiratory syndrome,Q18123741,P1542,Q81068910,2019–20 COVID-19 pandemic,has effect,Q103177,infectious disease,ast.wikipedia,Síndrome respiratoriu agudu grave,[[ast:Síndrome respiratoriu agudu grave|Síndro...,https://ast.wikipedia.org/wiki/Síndrome respir...
Q103177,severe acute respiratory syndrome,Q18123741,P1542,Q81068910,2019–20 COVID-19 pandemic,has effect,Q103177,infectious disease,azb.wikipedia,سارس,[[azb:سارس|سارس]],https://azb.wikipedia.org/wiki/سارس
Q103177,severe acute respiratory syndrome,Q18123741,P1542,Q81068910,2019–20 COVID-19 pandemic,has effect,Q103177,infectious disease,be.wikipedia,Цяжкі востры рэспіраторны сіндром,[[be:Цяжкі востры рэспіраторны сіндром|Цяжкі в...,https://be.wikipedia.org/wiki/Цяжкі востры рэс...
...,...,...,...,...,...,...,...,...,...,...,...,...
Q89713663,unknown,Q3241045,unknown,unknown,None,NaN,Q89713663,disease outbreak,pt.wikipedia,Pandemia de COVID-19 no Estado de São Paulo,[[pt:Pandemia de COVID-19 no Estado de São Pau...,https://pt.wikipedia.org/wiki/Pandemia de COVI...
Q89728160,2020 coronavirus pandemic in Fatehgarh Sahib d...,Q3241045,P361,Q81068910,2019–20 COVID-19 pandemic,part of,Q89728160,disease outbreak,NaN,NaN,NaN,NaN
Q89728354,2020 coronavirus pandemic in Kapurthala district,Q3241045,P361,Q81068910,2019–20 COVID-19 pandemic,part of,Q89728354,disease outbreak,NaN,NaN,NaN,NaN
Q89742590,2020 coronavirus pandemic in Maharajganj district,Q3241045,P361,Q81068910,2019–20 COVID-19 pandemic,part of,Q89742590,disease outbreak,NaN,NaN,NaN,NaN


In [54]:
# save nonHumans to pickle
nonHumansPages.to_pickle(f'pagesPerProjectNonHumans-{tNow()}.pickle')

In [55]:
import numpy as np
with open('pagesPerProjectNonHumans20200330.wikitext','w') as f:
    for project,data in nonHumansPages.groupby('project'):
        if project != np.nan:
            f.write('\n== %s == \n \n' % project )
            for wikilink,d  in data.groupby('wikilink'):                
                f.write('* %s (' % wikilink)
                for index,cause in d.iterrows():
                    if cause['connector Label'] == np.nan:
                        cause['connector Label'] == 'unknown'
                        cause['connected To Label'] == 'unknown'

                    output = '%s: %s, ' % (cause['connector Label'],cause['connected To Label'])
                output = output[0:-2]
                output += ')\n'
                f.write(output)

In [56]:
nonHumansPages[['project','page','url','wikilink']].drop_duplicates().to_excel(f'pagesPerProjectNonHumans{tNow()}.xls')

In [57]:
itemsInfoTablePages = itemsInfoTable.join(pagesPerProjectTable.set_index('wikidataItem'))

In [58]:
import numpy as np
with open('pagesPerProjectMethodologyMarch30.wikitext','w') as f:
    for project,data in nonHumansPages.groupby('project'):
        if project != np.nan:
            f.write('\n== %s == \n \n' % project )
            for wikilink,d  in data.groupby('wikilink'):                
                f.write('* %s (' % wikilink)
                for index,cause in d.iterrows():
                    if cause['connector Label'] != np.nan:
                        output = '%s: %s, ' % (cause['connector Label'],cause['connected To Label'])
                output = output[0:-2]
                output += ')\n'
                f.write(output)

# Human list by join pages & items

In [59]:
Humans  = itemsInfoTable[itemsInfoTable['Instace Of Label'] == 'human']

In [60]:
Humans

,item Label,Instace Of,connector,connected To,connected To Label,connector Label,item_id,Instace Of Label
Q5210439,Dale Folwell,Q5,P1050,Q84263196,COVID-19,medical condition,Q5210439,human
Q50634772,Isabel Díaz Ayuso,Q5,P1050,Q84263196,COVID-19,medical condition,Q50634772,human
Q28723650,Budi Karya Sumadi,Q5,P1050,Q84263196,COVID-19,medical condition,Q28723650,human
Q4662382,Aaron Rubashkin,Q5,P509,Q84263196,COVID-19,cause of death,Q4662382,human
Q88196412,António Vieira Monteiro,Q5,P509,Q84263196,COVID-19,cause of death,Q88196412,human
...,...,...,...,...,...,...,...,...
Q1675472,Marco Solari,Q5,P1050,Q84263196,COVID-19,medical condition,Q1675472,human
Q87975501,Narjes Khanalizadeh,Q5,P509,Q84263196,COVID-19,cause of death,Q87975501,human
Q87975501,Narjes Khanalizadeh,Q5,P1050,Q84263196,COVID-19,medical condition,Q87975501,human
Q7693568,Ted O'Brien,Q5,P1050,Q84263196,COVID-19,medical condition,Q7693568,human


In [61]:
HumansPages = Humans.join(pagesPerProjectTable.set_index('wikidataItem'))

In [62]:
HumansPages

,item Label,Instace Of,connector,connected To,connected To Label,connector Label,item_id,Instace Of Label,project,page,wikilink,url
Q1000366,Bucky Pizzarelli,Q5,P509,Q84263196,COVID-19,cause of death,Q1000366,human,azb.wikipedia,باکی پیزارلی,[[azb:باکی پیزارلی|باکی پیزارلی]],https://azb.wikipedia.org/wiki/باکی پیزارلی
Q1000366,Bucky Pizzarelli,Q5,P509,Q84263196,COVID-19,cause of death,Q1000366,human,bg.wikipedia,Бъки Пицарели,[[bg:Бъки Пицарели|Бъки Пицарели]],https://bg.wikipedia.org/wiki/Бъки Пицарели
Q1000366,Bucky Pizzarelli,Q5,P509,Q84263196,COVID-19,cause of death,Q1000366,human,commons.wikipedia,Category:Bucky Pizzarelli,[[commons:Category:Bucky Pizzarelli|Category:B...,https://commons.wikipedia.org/wiki/Category:Bu...
Q1000366,Bucky Pizzarelli,Q5,P509,Q84263196,COVID-19,cause of death,Q1000366,human,cs.wikipedia,Bucky Pizzarelli,[[cs:Bucky Pizzarelli|Bucky Pizzarelli]],https://cs.wikipedia.org/wiki/Bucky Pizzarelli
Q1000366,Bucky Pizzarelli,Q5,P509,Q84263196,COVID-19,cause of death,Q1000366,human,de.wikipedia,Bucky Pizzarelli,[[de:Bucky Pizzarelli|Bucky Pizzarelli]],https://de.wikipedia.org/wiki/Bucky Pizzarelli
...,...,...,...,...,...,...,...,...,...,...,...,...
Q999960,Jean-Laurent Cochet,Q5,P509,Q84263196,COVID-19,cause of death,Q999960,human,es.wikipedia,Jean-Laurent Cochet,[[es:Jean-Laurent Cochet|Jean-Laurent Cochet]],https://es.wikipedia.org/wiki/Jean-Laurent Cochet
Q999960,Jean-Laurent Cochet,Q5,P509,Q84263196,COVID-19,cause of death,Q999960,human,fr.wikipedia,Jean-Laurent Cochet,[[fr:Jean-Laurent Cochet|Jean-Laurent Cochet]],https://fr.wikipedia.org/wiki/Jean-Laurent Cochet
Q999960,Jean-Laurent Cochet,Q5,P509,Q84263196,COVID-19,cause of death,Q999960,human,pl.wikipedia,Jean-Laurent Cochet,[[pl:Jean-Laurent Cochet|Jean-Laurent Cochet]],https://pl.wikipedia.org/wiki/Jean-Laurent Cochet
Q999960,Jean-Laurent Cochet,Q5,P509,Q84263196,COVID-19,cause of death,Q999960,human,simple.wikipedia,Jean-Laurent Cochet,[[simple:Jean-Laurent Cochet|Jean-Laurent Coch...,https://simple.wikipedia.org/wiki/Jean-Laurent...


In [63]:
# save the humans to csv
HumansPages.to_csv-(f'humansPages{tNow()}.csv')

TypeError: unsupported operand type(s) for -: 'method' and 'str'

In [ ]:
HumansPages[['project','page','url','wikilink']].drop_duplicates().to_excel(f'pagesPerProjectHumans{tNow()}.xls')

# Failed: People Date Adding to HumansPages

In [ ]:
HumansPagesTemp = HumansPages

In [ ]:
import mwapi

def findWhenKeywordWasAddedWasAdded(page_name,project,date,keywords):
    """
    page_name: str, article title, ex: 'COVID-19'
    project: str, project id, ex: 'es.wikipedia'
    date: timestamp, counting from given day example '2020-01-01T00:00:00Z'
    keywords: list of str, ex: COVID or Q84263196
    """
    session = mwapi.Session("https://%s.org" % project, user_agent="dsaez@wikimedia.org - COVID-19 research")
    for response_doc in session.get(action='query', prop='revisions', titles=page_name, 
                                    rvprop=['ids', 'timestamp','content'], rvlimit=50, rvdir="newer", 
                                    formatversion=2,rvstart=date, continuation=True,):
        for rev_doc in response_doc['query']['pages'][0]['revisions']:
            rev_content = rev_doc['content'].lower()
            timestamp = rev_doc['timestamp']
            for keyword in keywords:
                if keyword.lower() in rev_content:
                    return timestamp
    return False

In [ ]:
def applyWithPageAndProject(page_name, project):
    try:
        timestamp = findWhenKeywordWasAddedWasAdded(page_name, project, '2020-01-01T00:00:00Z',['COVID','Coronavirus'])
    except:
        print('error', page_name)

In [ ]:
# Caution: Assign Project for reducing processing time
# HumansPagesTemp['Timestamp'] = HumansPagesTemp.apply(lambda x : applyWithPageAndProject(x['item Label'], x['project']), axis = 1)
# apply lamda + if -> https://datatofish.com/if-condition-in-pandas-dataframe/

In [ ]:
# HumansPagesWithDate = HumansPagesTemp

In [ ]:

# HumansPagesWithDate.to_csv('HumansPagesWithDate202004052037.csv')

In [ ]:
# HumansPagesWithDate

# Delete People views before the date

In [ ]:
viewsDeletedBeforeDate = viewsCorona